<a href="https://colab.research.google.com/github/ywang1110/NLP/blob/main/Build_a_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regular expression 

In [ ]:
import re
pattern = 'do you remember .*'   # '.*' -> catch-all. it means match any string of characters

In [ ]:
message = 'do you remember when you ate strawberries in the garden'
match = re.search(pattern, message)
if match:
  print('string matches!')

string matches!


In [ ]:
import re
pattern = "if (.*)"
message = "what would happen if bots took over the world"
match = re.search(pattern, message)

match.group(0)

'if bots took over the world'

In [ ]:
match.group(1)   # the group we defined by including the parentheses in the pattern

'bots took over the world'

## Grammatical transformation

In [ ]:
import re

def swap_pronouns(phrase):
  if 'I' in phrase:
    return re.sub('I', 'you', phrase)
  if 'my' in phrase:
    return re.sub('my', 'your', phrase)
  
  else:
    return phrase

swap_pronouns('I walk my dog')

'you walk my dog'

In [ ]:
pattern = 'do you remember (.*)'
message = 'do you remember when you ate strawberries in the garden'
phrase = re.search(pattern, message).group(1)
phrase

'when you ate strawberries in the garden'

## Extraction key phrases

In [ ]:
# example

import random

def match_rule(rules, message):
  response, phrase = 'default', None

  for pattern, responses in rules.items():
    match = re.search(pattern, message)
    if match is not None:
      response = random.choice(responses)
      if '{0}' in response:
        phrase = match.group(1)
  return response.format(phrase)    # '  {}'.format()

In [ ]:
# Test match_rule

rules = {'I want (.*)': ['What would it mean if you got {0}','Why do you want {0}',"What's stopping you from getting {0}"],
         'do you remember (.*)': ['Did you think I would forget {0}',"Why haven't you been able to forget {0}",'What about {0}','Yes .. and?']}
match_rule(rules, 'do you remember your last birthday')

'Yes .. and?'

## Pronouns

In [ ]:
def replace_pronouns(message):

  message = message.lower()
  if 'me' in message:
    return re.sub('I', 'you', message)
  if '' in message:
    return re.sub('my', 'your', message)
  if 'your' in message:
    return re.sub('your', 'me', message)
  return message

In [ ]:
print(replace_pronouns("my last birthday"))
print(replace_pronouns("when you went to Florida"))
print(replace_pronouns("I had my own castle"))
print(replace_pronouns('I walk my dog'))

your last birthday
when you went to florida
i had your own castle
i walk your dog


## Intergrate together


In [ ]:
def match_rule(rules, message):
  response, phrase = 'default', None

  for pattern, responses in rules.items():
    match = re.search(pattern, message)
    if match is not None:
      response = random.choice(responses)
      if '{0}' in response:
        phrase = match.group(1)
  return response, phrase

In [ ]:
# Test match_rule

rules = {'I want (.*)': ['What would it mean if you got {0}','Why do you want {0}',"What's stopping you from getting {0}"],
         'do you remember (.*)': ['Did you think I would forget {0}',"Why haven't you been able to forget {0}",'What about {0}','Yes .. and?']}
match_rule(rules, 'do you remember your last birthday')

('What about {0}', 'your last birthday')

In [ ]:
def replace_pronouns(message):
  message = message.lower()
  if 'me' in message:
    return re.sub('me', 'you', message)
  elif 'my' in message:
    return re.sub('my', 'your', message)
  elif 'your' in message:
    return re.sub('your', 'my', message)
  elif 'you' in message:
    return re.sub('you', 'me', message)

In [ ]:
# Test replace_pronous

print(replace_pronouns('I walk my dog'))

i walk your dog


In [ ]:
# integret

def respond(message):
  response, phrase = match_rule(rules, message)
  if '{0}' in response:
    # Replace the pronous in the phrase
    phrase = replace_pronouns(phrase)
    # Include the phrase in the response 
    response = response.format(phrase)
  return response

user_template = 'USER : {0}'
bot_template = 'BOT : {0}'


def send_message(message):
    print(user_template.format(message))
    response = respond(message)
    print(bot_template.format(response))

In [ ]:
send_message("do you remember your last birthday")

USER : do you remember your last birthday
BOT : Did you think I would forget my last birthday


# NLU (Natrual Language Understanding)
- ** Convert freeform text into structured data within a particular domain**
  - A restaurant booking bot is **able to understand a sentence like** 'I am looking for Mexican restaurant in the centre of the town' and **query a database** or **API** to find a matching results

    - **Intent:** restaurant_search
    - **Entities:** Book a table for **June 10th** at a **sushi** restautant in **New York** City  
      - **NER:** Named Entity Recognition (aims to find more universal entities like names of people, organizations, dates, etc.)
      - **'Bots'**: want a narrower definiation of your entities that are specific to your domain

## Regular expression to detect **intent** and **entities**

### '|' =OR

In [ ]:
re.search(r"(hello|hi|hey)", 'hey there!') is not None

True

In [ ]:
re.search(r"(hello|hi|hey)", 'which one') is not None  # hi also matches 'which'

True

### Word bounary \b

In [ ]:
re.search(r"\b(hello|hi|hey)\b", 'which one') is not None

False

### Pattern multiple times (re.compile)

In [ ]:
# Find all captitalized words in a sentence
pattern = re.compile('[A-Z]{1}[a-z]*')  
# []-> A range of characters
# [A-Z]{1} -> exactly 1 captial letter
# * -> 0 or more of occurances of lower character

In [ ]:
message = """
Mary is a friend of mine, 
she studies at Oxford and 
now works at Google"""

pattern.findall(message)

['Mary', 'Oxford', 'Google']

### str join()
 - The join() method is a string method and returns a string in which the elements of sequence have been joined by str separator.

In [ ]:
greet = ['hey', 'hi', 'hello']
'|'.join(greet)

'hey|hi|hello'

In [ ]:
keywords = {'goodbye': ['bye', 'farewell'],
 'greet': ['hello', 'hi', 'hey'],
 'thankyou': ['thank', 'thx']}

In [ ]:
patterns = {}

for intent, keys in keywords.items():
  patterns[intent] = re.compile('|'.join(keys))
print(patterns)

{'goodbye': re.compile('bye|farewell'), 'greet': re.compile('hello|hi|hey'), 'thankyou': re.compile('thank|thx')}


### Integret 

In [ ]:
# Define a function to find the intent of a message

def match_intent(message):
  matched_intent = None
  for intent, pattern in patterns.items():
    re.search(pattern, message) is not None
    matched_intent = intent
  return matched_intent

In [ ]:
# Define a respond function

responses = {'default': 'default message',
 'goodbye': 'goodbye for now',
 'greet': 'Hello you! :)',
 'thankyou': 'you are very welcome'}

def respond(message):
   intent = match_intent(message)
   if intent in responses:
     key = intent
     return responses[key]

In [ ]:
send_message("bye byeee")

USER : bye byeee
BOT : you are very welcome
